In [1]:
import carla
import math
import random
import time
import numpy as np
import cv2
import subprocess

In [2]:
# Create a new client instance
client = carla.Client('localhost', 2000)
world = client.get_world()

bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

vehicle_bp = bp_lib.find('vehicle.dodge.charger')
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

spectator = world.get_spectator()
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4, z=2.5)), vehicle.get_transform().rotation)
spectator.set_transform(transform)

In [3]:
# Start the traffic light controller in the background
traffic_light_process = subprocess.Popen(['python', 'traffic_light_controller.py'])
traffic_process = subprocess.Popen(['python', 'generate_traffic.py', '--number-of-vehicles', '30', '--number-of-walkers', '50'])
time.sleep(5)

# Set up traffic manager
traffic_manager = client.get_trafficmanager()
vehicle.set_autopilot(True, traffic_manager.get_port())

In [4]:
camera_init_trans = carla.Transform(carla.Location(x=0.0, y=0.0, z=1.6), carla.Rotation(pitch=0.0))

def set_camera_attributes(camera_bp):
    # Default settings for all cameras
    camera_bp.set_attribute('image_size_x', '800')
    camera_bp.set_attribute('image_size_y', '600')
    camera_bp.set_attribute('fov', '90')

# RGB camera
camera_bp = bp_lib.find('sensor.camera.rgb')
set_camera_attributes(camera_bp)
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

# Semantic camera
sem_camera_bp = bp_lib.find('sensor.camera.semantic_segmentation')
set_camera_attributes(sem_camera_bp)
sem_camera = world.spawn_actor(sem_camera_bp, camera_init_trans, attach_to=vehicle)

# Instance Segmentation camera
inst_camera_bp = bp_lib.find('sensor.camera.instance_segmentation')
set_camera_attributes(inst_camera_bp)
inst_camera = world.spawn_actor(inst_camera_bp, camera_init_trans, attach_to=vehicle)

# Monocular Depth camera
depth_camera_bp = bp_lib.find('sensor.camera.depth')
set_camera_attributes(depth_camera_bp)
depth_camera = world.spawn_actor(depth_camera_bp, camera_init_trans, attach_to=vehicle)


In [5]:
# Initialize sensor data storage BEFORE calling `listen()`
image_w = 800
image_h = 600

sensor_data = {
    'rgb_image': np.zeros((image_h, image_w, 4), dtype=np.uint8),
    'sem_image': np.zeros((image_h, image_w, 4), dtype=np.uint8),
    'depth_image': np.zeros((image_h, image_w, 4), dtype=np.uint8),
    'inst_image': np.zeros((image_h, image_w, 4), dtype=np.uint8),
}

def add_label(image, label, position=(10, 50), font_scale=0.8, color=(255, 255, 255)):
    """Adds a text label to an image."""
    labeled_image = image.copy()  # Work on a copy to avoid modifying original
    cv2.putText(labeled_image, label, position, cv2.FONT_HERSHEY_SIMPLEX, 
                font_scale, color, 2, cv2.LINE_AA)
    return labeled_image  # Return the labeled image

def rgb_callback(image, data_dict):
    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    add_label(img, "RGB Camera")
    data_dict['rgb_image'] = img

def sem_callback(image, data_dict):
    image.convert(carla.ColorConverter.CityScapesPalette)
    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    add_label(img, "Semantic Segmentation")
    data_dict['sem_image'] = img

def inst_callback(image, data_dict):
    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    add_label(img, "Instance Segmentation")
    data_dict['inst_image'] = img

def depth_callback(image, data_dict):
    image.convert(carla.ColorConverter.LogarithmicDepth)
    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    add_label(img, "Depth Camera")
    data_dict['depth_image'] = img

# Start sensor listeners
camera.listen(lambda image: rgb_callback(image, sensor_data))
sem_camera.listen(lambda image: sem_callback(image, sensor_data))
inst_camera.listen(lambda image: inst_callback(image, sensor_data))
depth_camera.listen(lambda image: depth_callback(image, sensor_data))

In [8]:
# Create display windows
cv2.namedWindow('All Cameras', cv2.WINDOW_NORMAL)
cv2.resizeWindow('All Cameras', 1280, 960)

cv2.namedWindow('Duplicate RGB', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Duplicate RGB', 800, 600)  # Keep RGB window smaller

def preprocess_image(img):
    """Ensure all images have 3 channels (convert to RGB if needed)."""
    if img.shape[2] == 4:  # If image has 4 channels (RGBA)
        img = img[:, :, :3]  # Drop the alpha channel
    return img

while True:
    try:
        # Convert all sensor images to 3-channel RGB format
        sensor_data_processed = {key: preprocess_image(img) for key, img in sensor_data.items()}

        # Check if all images are available before displaying
        if any(img.shape[0] == 0 for img in sensor_data_processed.values()):
            print("Waiting for camera feeds...")
            continue

        # Get the RGB image **without** label for the Duplicate RGB window
        rgb_clean = sensor_data_processed['rgb_image'].copy()

        # Create a **labeled** RGB image for the "All Cameras" window
        rgb_labeled = add_label(sensor_data_processed['rgb_image'], "RGB Camera")

        # Concatenate images in a 2-row format for display
        top_row = np.concatenate([
            rgb_labeled,  # Labeled RGB Image for "All Cameras"
            add_label(sensor_data_processed['sem_image'], "Semantic Segmentation")
        ], axis=1)
        
        lower_row = np.concatenate([
            add_label(sensor_data_processed['depth_image'], "Depth Camera"),
            add_label(sensor_data_processed['inst_image'], "Instance Segmentation")
        ], axis=1)

        # Combine both rows into a single tiled view
        tiled = np.concatenate((top_row, lower_row), axis=0)

        # Display the combined camera feeds in the main window
        cv2.imshow('All Cameras', tiled)

        # Display the clean RGB image in the Duplicate RGB window
        cv2.imshow("Duplicate RGB", rgb_clean)

        # Exit on pressing 'q'
        if cv2.waitKey(1) == ord('q'):
            break
    except Exception as e:
        print(f"Error displaying images: {e}")

# Cleanup: Close all OpenCV windows
cv2.destroyAllWindows()